데이터 개수 확인

In [6]:
import os

def count_files_in_directory(directory):
    total_files = 0
    for root, dirs, files in os.walk(directory):
        total_files += len(files)
    return total_files

directory_path = "/home/addinedu/dev_ws/data/handwritten_dataset/train/x"
total_files = count_files_in_directory(directory_path)
print("폴더 내 데이터 개수:", total_files)


폴더 내 데이터 개수: 400


리사이즈

In [15]:
import os
import cv2

# 원본 이미지가 있는 폴더 경로
source_folder = "/home/addinedu/dev_ws/data/X"

# 새로운 폴더 경로
destination_folder = "/home/addinedu/dev_ws/data/new_0"

# 새로운 폴더가 없다면 생성
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# 폴더 내의 모든 파일 목록 가져오기
file_list = os.listdir(source_folder)

# 모든 파일에 대해 작업 수행
for file_name in file_list:
    # 파일의 전체 경로 구성
    source_file_path = os.path.join(source_folder, file_name)
    
    # 이미지 로드
    image = cv2.imread(source_file_path)
    
    # 이미지가 None이 아니라면 (올바르게 로드되었다면)
    if image is not None:
        # 이미지를 32x32 픽셀로 리사이즈
        resized_image = cv2.resize(image, (32, 32))
        
        # 새로운 파일 경로 구성
        destination_file_path = os.path.join(destination_folder, file_name)
        
        # 리사이즈된 이미지를 새로운 폴더에 저장
        cv2.imwrite(destination_file_path, resized_image)


+-15 회전시켜서 데이터 증강


In [23]:
import cv2
import os

# 입력 이미지 폴더 경로
input_directory = "/home/addinedu/dev_ws/data/handwritten_dataset/train/x"
# 출력 이미지 폴더 경로
output_directory = "/home/addinedu/dev_ws/data/handwritten_dataset/train/x_new_train1000"
# 이미지 확장자
image_extension = ".jpg"

# 이미지 회전 각도 범위
rotation_angles = range(-15, 16)

# 입력 이미지 폴더가 없다면 생성
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# 입력 폴더 내의 이미지 파일들에 대해 반복
for filename in os.listdir(input_directory):
    if filename.endswith(image_extension):
        # 이미지 파일 경로
        image_path = os.path.join(input_directory, filename)
        # 이미지 로드
        image = cv2.imread(image_path)
        # 이미지 회전 및 저장
        for angle in rotation_angles:
            # 회전 변환 매트릭스 계산
            rotation_matrix = cv2.getRotationMatrix2D((image.shape[1] / 2, image.shape[0] / 2), angle, 1)
            # 이미지 회전
            rotated_image = cv2.warpAffine(image, rotation_matrix, (image.shape[1], image.shape[0]))
            # 새로운 파일명 생성
            output_filename = os.path.splitext(filename)[0] + "_rotated_" + str(angle) + image_extension
            # 새로운 파일 경로
            output_path = os.path.join(output_directory, output_filename)
            # 이미지 저장
            cv2.imwrite(output_path, rotated_image)

print("이미지 회전 및 저장이 완료되었습니다.")


이미지 회전 및 저장이 완료되었습니다.


지정영역 제외하고 흰색으로 만들기

In [26]:
import cv2
import os
import numpy as np

# 입력 이미지 폴더 경로
input_directory = "/home/addinedu/dev_ws/data/handwritten_dataset/train/x_new_train1000"
# 출력 이미지 폴더 경로
output_directory = "/home/addinedu/dev_ws/data/handwritten_dataset/train/a_x_new_train1000"
# 제외할 영역의 좌상단과 우하단 좌표
exclude_area = [(5, 5), (25, 25)]  # (x1, y1), (x2, y2)

# 입력 이미지 폴더가 없다면 생성
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# 입력 폴더 내의 이미지 파일들에 대해 반복
for filename in os.listdir(input_directory):
    # 이미지 파일 경로
    image_path = os.path.join(input_directory, filename)
    # 이미지 로드
    image = cv2.imread(image_path)
    # 이미지 크기
    height, width, _ = image.shape
    # 제외할 영역에 대한 마스크 생성
    mask = np.zeros((height, width), dtype=np.uint8)
    cv2.rectangle(mask, exclude_area[0], exclude_area[1], (255), thickness=cv2.FILLED)
    # 이미지를 흰색으로 채우되, 제외 영역은 원본 유지
    result = np.where(mask[:, :, None] == 255, image, 255)
    # 출력 이미지 경로
    output_path = os.path.join(output_directory, filename)
    # 이미지 저장
    cv2.imwrite(output_path, result)

print("이미지 처리가 완료되었습니다.")


이미지 처리가 완료되었습니다.


데이터 라벨링 후 .csv로 저장

In [8]:
import os
import pandas as pd

# 이미지 폴더 경로
image_folder = "/home/addinedu/dev_ws/data/images.cv_cda9n6fio287zp8fuwsivo/data/new_train1000"
# 라벨
label = "0"

# 이미지 파일 목록
image_files = os.listdir(image_folder)

# 빈 리스트 생성
data = []

# 이미지 파일에 대해 반복하면서 라벨링
for image_file in image_files:
    # 이미지 파일 경로
    image_path = os.path.join(image_folder, image_file)
    # 이미지 파일 경로와 라벨을 튜플로 묶어 리스트에 추가
    data.append((image_path, label))

# 리스트를 데이터프레임으로 변환
df = pd.DataFrame(data, columns=["Image_Path", "Label"])

# 라벨링된 데이터를 CSV 파일로 저장
output_csv_path = "/home/addinedu/dev_ws/data/images.cv_cda9n6fio287zp8fuwsivo/data/new_train1000.csv"
df.to_csv(output_csv_path, index=False)

print("라벨링이 완료되었습니다. 라벨링된 데이터는", output_csv_path, "에 저장되었습니다.")


라벨링이 완료되었습니다. 라벨링된 데이터는 /home/addinedu/dev_ws/data/images.cv_cda9n6fio287zp8fuwsivo/data/new_train1000.csv 에 저장되었습니다.


라벨링2

In [4]:
import os
import pandas as pd

# 이미지 파일이 있는 폴더 경로
image_folder = "/home/addinedu/dev_ws/data/ox_images/test"

# 이미지 파일 목록
image_files = os.listdir(image_folder)

# 라벨과 이미지 파일명을 저장할 리스트
labels = []
file_names = []

# 각 이미지 파일의 첫 글자를 라벨로 하여 저장
for image_file in image_files:
    label = image_file[0]  # 파일명의 첫 글자를 라벨로 설정
    labels.append(label)
    file_names.append(image_file)

# 데이터프레임 생성
data = pd.DataFrame({"File_Name": file_names, "Label": labels})

# CSV 파일로 저장
output_csv_path = "/home/addinedu/dev_ws/data/ox_images/test_labels.csv"
data.to_csv(output_csv_path, index=False)

print("라벨링된 데이터가 CSV 파일로 저장되었습니다:", output_csv_path)


라벨링된 데이터가 CSV 파일로 저장되었습니다: /home/addinedu/dev_ws/data/ox_images/train_labels.csv


라벨링3

In [2]:
import os
import pandas as pd

# 이미지 파일이 있는 폴더 경로
image_folder = "/home/addinedu/dev_ws/data/ox_images/test"

# 이미지 파일 목록
image_files = os.listdir(image_folder)

# 라벨과 이미지 파일명을 저장할 리스트
labels = []
file_names = []

# 각 이미지 파일의 첫 글자를 라벨로 하여 저장
for image_file in image_files:
    label = 0 if image_file[0] == 'O' else 1
    labels.append(label)
    file_names.append(image_file)

# 데이터프레임 생성
data = pd.DataFrame({"File_Name": file_names, "Label": labels})

# CSV 파일로 저장
output_csv_path = "/home/addinedu/dev_ws/data/ox_images/test_labels.csv"
data.to_csv(output_csv_path, index=False)

print("라벨링된 데이터가 CSV 파일로 저장되었습니다:", output_csv_path)


라벨링된 데이터가 CSV 파일로 저장되었습니다: /home/addinedu/dev_ws/data/ox_images/test_labels.csv


라벨링확인2


In [6]:
import os
import cv2
import random
import pandas as pd

# 이미지 파일이 있는 폴더 경로
image_folder = "/home/addinedu/dev_ws/data/ox_images/train"
# 라벨 CSV 파일 경로
label_csv_path = "/home/addinedu/dev_ws/data/ox_images/train_labels.csv"

# 라벨 CSV 파일 로드
label_df = pd.read_csv(label_csv_path)

# 이미지 파일 목록
image_files = os.listdir(image_folder)

# 이미지와 라벨을 랜덤으로 선택
random_index = random.randint(0, len(image_files) - 1)
image_file = image_files[random_index]
label = label_df.loc[label_df['File_Name'] == image_file, 'Label'].iloc[0]

# 이미지 로드
image_path = os.path.join(image_folder, image_file)
image = cv2.imread(image_path)

# 이미지와 라벨을 함께 표시
cv2.putText(image, label, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
cv2.imshow("Image with Label", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


라벨링 확인

In [1]:
import pandas as pd

# 라벨링된 CSV 파일 경로
csv_file_path = "/home/addinedu/dev_ws/data/ox_images/train_labels.csv"

# CSV 파일 로드
df = pd.read_csv(csv_file_path)

# 데이터프레임 출력
print(df.head())


   File_Name Label
0  x_311.jpg     x
1   O_76.jpg     O
2  x_131.jpg     x
3  O_248.jpg     O
4  x_120.jpg     x
